<a href="https://colab.research.google.com/github/saniagonsalves/DM_ANN/blob/main/state_of_art_libraries_for_Approximate_nearest_neighbor_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1.LSH**

**To make recommendations on conference papers by using LSH to quickly query all of the known conference papers**

In [ ]:
!pip install datasketch

In [ ]:
pip install --upgrade pandas

In [ ]:
import pandas.core.indexes as i

In [ ]:
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'  
!pip install annoy

In [ ]:
import numpy as np
import pandas as pd
import re
import time
from datasketch import MinHash, MinHashLSHForest

**Data Preprocessing**

*   Remove all punctuation.
*   Lowercase all text.
*   Create unigram shingles (tokens) by separating any white space.


In [ ]:
#Preprocess will split a string of text into individual tokens/shingles based on whitespace.
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

example of the preprocessing step below

In [ ]:
text = 'The devil went down to California'
print('The shingles (tokens) are:', preprocess(text))

The shingles (tokens) are: ['the', 'devil', 'went', 'down', 'to', 'california']


we will use the standard number of permutations of 128. We will also start by just making one recommendation.

In [ ]:
#Number of Permutations
permutations = 128

#Number of Recommendations to return
num_recommendations = 1

To create the Minhash Forest


*   Pass in a dataframe with every string to query
*   Preprocess a string of text using the preprocessing step above
*   Set the number of permutations in the MinHash
*   MinHash the string on all of the shingles in the string
*   Build a forest of all the MinHashed strings
*   Index the forest to make it searchable



In [ ]:
def get_forest(data, perms):
    start_time = time.time()
    
    minhash = []
    
    for text in data['text']:
        tokens = preprocess(text)
        m = MinHash(num_perm=perms)
        for s in tokens:
            m.update(s.encode('utf8'))
        minhash.append(m)
        
    forest = MinHashLSHForest(num_perm=perms)
    
    for i,m in enumerate(minhash):
        forest.add(i,m)
        
    forest.index()
    
    print('It took %s seconds to build forest.' %(time.time()-start_time))
    
    return forest


* Preprocess the text into shingles.
* Set the same number of permutations for the MinHash that was used to build the forest.
* Create the MinHash on the text using all the shingles.
* Query the forest with MinHash and return the number of requested recommendations.
* Provide the titles of each conference paper recommended.

In [ ]:
def predict(text, database, perms, num_results, forest):
    start_time = time.time()
    
    tokens = preprocess(text)
    m = MinHash(num_perm=perms)
    for s in tokens:
        m.update(s.encode('utf8'))
        
    idx_array = np.array(forest.query(m, num_results))
    if len(idx_array) == 0:
        return None # if your query is empty, return none
    
    result = database.iloc[idx_array]['title']
    
    print('It took %s seconds to query forest.' %(time.time()-start_time))
    
    return result

* load the CSV containing all the conference papers and create a new field that combines the title and the abstract into one field, so that shingles using both title and abstract are built.
* query any string of text such as a title or general topic to return a list of recommendations

In [ ]:
db = pd.read_csv('/content/papers.csv')
db['text'] = db['title'] + ' ' + db['abstract']
forest = get_forest(db, permutations)

It took 20.31286120414734 seconds to build forest.


In [ ]:
num_recommendations = 10
title = 'Using a neural net to instantiate a deformable model'
result = predict(title, db, permutations, num_recommendations, forest)
print('\n Top Recommendation(s) is(are) \n', result)

It took 0.01011204719543457 seconds to query forest.

 Top Recommendation(s) is(are) 
 450     Speech Production Using A Neural Network with ...
995     Neural Network Weight Matrix Synthesis Using O...
5       Using a neural net to instantiate a deformable...
5191    A Self-Organizing Integrated Segmentation and ...
7       ICEG Morphology Classification using an Analog...
3056    Proximity Effect Corrections in Electron Beam ...
112     Adaptive Neural Networks Using MOS Charge Storage
2069    Analytic Solutions to the Formation of Feature...
7094    Non-Intrusive Gaze Tracking Using Artificial N...
2457    Inferring Neural Firing Rates from Spike Train...
Name: title, dtype: object


In [ ]:
!pip install faiss

     |████████████████████████████████| 4.7 MB 5.3 MB/s 


In [ ]:
!pip install lightfm

     |████████████████████████████████| 310 kB 5.2 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705334 sha256=19acb548830ca22e8c7b23a7ecfb9fe3d02aded3cb6ca4265271641ce2caa61a
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [ ]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import pickle

In [ ]:
movielens = fetch_movielens()
train = movielens['train']
test = movielens['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=movielens['item_features'], epochs=20 )

item_vectors = movielens['item_features'] * model.item_embeddings

In [ ]:
with open('movies.pickle', 'wb') as f:
    pickle.dump({"name": movielens['item_labels'], "vector": item_vectors}, f)

In [ ]:
import pickle

In [ ]:
def load_data():
    with open('movies.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[ 0.39782977,  0.11020264,  0.1589934 , ..., -0.11217553,
          0.07283162, -0.03622515],
        [ 0.02751156, -0.12293395, -0.41641998, ...,  0.01213327,
          0.10646569,  0.12970205],
        [ 0.21022919,  0.05421719,  0.17373037, ...,  0.263918  ,
          0.10492586, -0.12712367],
        ...,
        [-0.12282974,  0.09805849, -0.10066735, ...,  0.03841189,
         -0.05532929,  0.02093315],
        [-0.07288302, -0.1007875 ,  0.02831026, ..., -0.10820378,
         -0.13741872, -0.14620647],
        [-0.01425009,  0.0023182 , -0.09769698, ..., -0.02008126,
         -0.14971486,  0.04387938]], dtype=float32)}

**2)Exhaustive Search**

In [ ]:
!pip install faiss-gpu

     |████████████████████████████████| 89.7 MB 6.7 kB/s 


In [ ]:
import faiss
import pickle


In [ ]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors.shape[1])
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = BruteForceIndex(data["vector"], data["name"])

In [ ]:
movie_vector, movie_name = data['vector'][90:91], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Akira (1988)
* Bram Stoker's Dracula (1992)
* Heavy Metal (1981)
* Sword in the Stone, The (1963)
* 20,000 Leagues Under the Sea (1954)
* Fantasia (1940)
* Casper (1995)
* Ghost in the Shell (Kokaku kidotai) (1995)
* Sound of Music, The (1965)


**3) product quantization**

In [ ]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Pink Floyd - The Wall (1982)',
 'Aladdin (1992)',
 'Mary Poppins (1964)',
 'Fox and the Hound, The (1981)',
 'Fantasia (1940)',
 '20,000 Leagues Under the Sea (1954)',
 'Sword in the Stone, The (1963)',
 'Sound of Music, The (1965)',
 'Old Yeller (1957)']

**4) trees and graphs**

In [ ]:
import annoy
class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Fantasia (1940)
* Heavy Metal (1981)
* Beauty and the Beast (1991)
* Lion King, The (1994)
* Star Trek: The Wrath of Khan (1982)
* Jurassic Park (1993)
* Batman (1989)
* Aladdin (1992)
* E.T. the Extra-Terrestrial (1982)


In [ ]:
!pip install nmslib

     |████████████████████████████████| 13.5 MB 75 kB/s 
     |████████████████████████████████| 188 kB 49.8 MB/s 


In [ ]:
import nmslib

In [ ]:
class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Bram Stoker's Dracula (1992)
* Fantasia (1940)
* Heavy Metal (1981)
* Beauty and the Beast (1991)
* Lion King, The (1994)
* Star Trek: The Wrath of Khan (1982)
* Akira (1988)
* Jurassic Park (1993)
* Sound of Music, The (1965)


**5) hnsw**

In [ ]:
import shutil
import urllib.request as request
from contextlib import closing

# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [ ]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [ ]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [ ]:
# data we will search through
wb = read_fvecs('/content/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('/content/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

**To explore the implementation of HNSW in Facebook AI Similarity Search (Faiss)**

In [ ]:
import faiss
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

# 1M samples
xb = read_fvecs('/content/sift_base.fvecs')
# queries
xq = read_fvecs('/content/sift_query.fvecs')[0].reshape(1, -1)
xq_full = read_fvecs('/content/sift_query.fvecs')

In [ ]:
xq.shape

(1, 128)

In [ ]:
wb.shape

(1000000, 128)

In [ ]:
# setup our HNSW parameters
d = 128  # vector size
M = 32
efSearch = 32  # number of entry points (neighbors) we use on each layer
efConstruction = 32  # number of entry points used on each layer
                     # during construction

index = faiss.IndexHNSWFlat(d, M)
print(index.hnsw)

<faiss.swigfaiss.HNSW; proxy of <Swig Object of type 'faiss::HNSW *' at 0x7ff2f9ba63f0> >


Before building the index with index.add the HNSW structure is empty:

In [ ]:
# the HNSW index starts with no levels
index.hnsw.max_level

-1

In [ ]:
# and levels (or layers) are empty too
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([], dtype=int64)

We can set the efConstruction and efSearch parameters, only efConstruction must be set before building the index. efSearch only affects search time behavior.

In [ ]:
index.hnsw.efConstruction = efConstruction
index.hnsw.efSearch = efSearch

In [ ]:
index.add(xb)

Now that we have added our data (and built the index) we will see that the HNSW structure has been populated.



In [ ]:
# after adding our data we will find that the level
# has been set automatically
index.hnsw.max_level

4

In [ ]:
# and levels (or layers) are now populated
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

In [ ]:
index.hnsw.entry_point

118295

The HNSW:set_default_probas function (from HNSW.cpp)calculates the number of neighbors (in total) a vertex will have across the calculated number of layers. We find that Faiss' implementation does not use M_max or M_max0 directly, but instead uses M to set these values. M_max is set to M, and M_max is set to 2*M.

In [ ]:
def set_default_probas(M: int, m_L: float):
    nn = 0  # set nearest neighbors count = 0
    cum_nneighbor_per_level = []
    level = 0  # we start at level 0
    assign_probas = []
    while True:
        # calculate probability for current level
        proba = np.exp(-level / m_L) * (1 - np.exp(-1 / m_L))
        # once we reach low prob threshold, we've created enough levels
        if proba < 1e-9: break
        assign_probas.append(proba)
        # neighbors is == M on every level except level 0 where == M*2
        nn += M*2 if level == 0 else M
        cum_nneighbor_per_level.append(nn)
        level += 1
    return assign_probas, cum_nneighbor_per_level

In [ ]:
assign_probas, cum_nneighbor_per_level = set_default_probas(
    32, 1/np.log(32)
)
assign_probas, cum_nneighbor_per_level

([0.96875,
  0.030273437499999986,
  0.0009460449218749991,
  2.956390380859371e-05,
  9.23871994018553e-07,
  2.887099981307982e-08],
 [64, 96, 128, 160, 192, 224])

In [ ]:
# this is copy of HNSW::random_level function
def random_level(assign_probas: list, rng):
    # get random float from 'r'andom 'n'umber 'g'enerator
    f = rng.uniform() 
    for level in range(len(assign_probas)):
        # if the random float is less than level probability...
        if f < assign_probas[level]:
            # ... we assert at this level
            return level
        # otherwise subtract level probability and try again
        f -= assign_probas[level]
    # below happens with very low probability
    return len(assign_probas) - 1

In [ ]:
chosen_levels = []
rng = np.random.default_rng(12345)
for _ in range(1_000_000):
    chosen_levels.append(random_level(assign_probas, rng))
np.bincount(chosen_levels)

array([968821,  30170,    985,     23,      1])

In [ ]:
1/np.log(32)  # the previous value we used for m_L

0.28853900817779266

In [ ]:
set_default_probas(32, 0.09)

([0.9999850546614752, 1.4945115161637832e-05], [64, 96])

In [ ]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

In [ ]:
del index
index = faiss.IndexHNSWFlat(d, 32)
index.hnsw.set_default_probas(32, 0.09)  # HNSW::set_default_probas(int M, float levelMult)
index.hnsw.efConstruction = efConstruction
index.add(xb)

In [ ]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([     0, 968746,  30276,    951,     26,      1])

Finally, let's validate that m_L values ~0 produce a single layer HNSW graph 

In [ ]:
assign_probas, cum_nneighbor_per_level = set_default_probas(32, 0.0000001)
assign_probas, cum_nneighbor_per_level

([1.0], [64])

In [ ]:
chosen_levels = []
rng = np.random.default_rng(12345)
for _ in range(1_000_000):
    chosen_levels.append(random_level(assign_probas, rng))

In [ ]:
np.bincount(chosen_levels)

array([1000000])

Faiss also always ensures that at least one vertex is included at the highest level, as we can see by creating a small index:

In [ ]:
del index
index = faiss.IndexHNSWFlat(d, 32)
index.hnsw.efConstruction = efConstruction
index.add(xb[:1_000])

In [ ]:
levels = faiss.vector_to_array(index.hnsw.levels)
np.bincount(levels)

array([  0, 974,  25,   1])